## Librerías 

In [1]:
import numpy as np
import joblib ### para cargar array

########Paquetes para NN #########
import tensorflow as tf
from sklearn import metrics ### para analizar modelo
from sklearn.ensemble import RandomForestClassifier  ### para analizar modelo
import pandas as pd
from sklearn.utils import class_weight
from sklearn import tree


import cv2 ### para leer imagenes jpeg
### pip install opencv-python

from matplotlib import pyplot as plt #

### Preprocesamiento

In [2]:
### cargar bases_procesadas ####

x_train = joblib.load('salidas/x_train.pkl')
y_train = joblib.load('salidas/y_train.pkl')
x_test = joblib.load('salidas/x_test.pkl')
y_test = joblib.load('salidas/y_test.pkl')



In [3]:
x_train.max()

255

In [4]:
x_test.max()

255

In [5]:
#### Escalar ######################
x_train = x_train.astype('float32') ## para poder escalarlo
x_test = x_test.astype('float32') ## para poder escalarlo
x_train /= 255 ### escalaro para que quede entre 0 y 1
x_test /= 255

###### verificar tamaños
print(x_train.shape)
x_test.shape

(2372, 100, 100, 3)


(336, 100, 100, 3)

In [6]:
np.product(x_train[1].shape) ## cantidad de variables por imagen

30000

In [7]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([1569,  803], dtype=int64))

In [8]:
np.unique(y_test, return_counts=True)

(array([0, 1], dtype=int64), array([208, 128], dtype=int64))

In [9]:
##### convertir a 1 d array ############
x_train2 = x_train.reshape(2372,30000)
x_test2 = x_test.reshape(336, 30000)
print(x_train2.shape)
x_test2.shape

(2372, 30000)


(336, 30000)

### Selección de métricas 

AUC:

Recall: Se selecciona el Recall como métrica de rendimiento para nuestro modelo de predicción de cáncer de mama porque queremos minimizar los falsos negativos (casos en los que el modelo predice incorrectamente que una persona no tiene cáncer cuando en realidad sí lo tiene). El Recall mide la proporción de casos positivos correctamente identificados, asegurando que detectemos la mayor cantidad posible de personas con cáncer, lo cual es esencial en aplicaciones de salud donde un diagnóstico omitido puede tener consecuencias graves.

## 1. Modelo Random Forest

In [ ]:
rf=RandomForestClassifier()
rf.fit(x_train2, y_train)

pred_train=rf.predict(x_train2)
r_metric=metrics.recall_score(y_train, pred_train)
auc_metric=metrics.roc_auc_score(y_train, pred_train)

metricas_train = pd.DataFrame({
    'Metric': ['Recall', 'AUC'],
    'Score': [r_metric, auc_metric]
})

print(metricas_train)

In [ ]:
pred_test=rf.predict(x_test2)
r_metric=metrics.recall_score(y_test, pred_test)
auc_metric=metrics.roc_auc_score(y_test, pred_test)

metricas_test = pd.DataFrame({
    'Metric': ['Recall', 'AUC'],
    'Score': [r_metric, auc_metric]
})

print(metricas_test)

## 2. Redes neuronales (estándar)

In [ ]:
y_train = y_train.ravel() #convierte y_train en un arreglo unidimensional


# Definir el modelo de red neuronal con regularización Dropout
fc_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=x_train.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Dropout para reducir el sobreajuste; 'apaga' el 30% de neuronas
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\delva\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
#Configurar el optimizador
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) #busca minimizar la función de pérdida del modelo


In [12]:
# Calcular los pesos de cada clase para mejorar el aprendizaje de las clases desbalanceadas
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

#Compilar el modelo
fc_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['Recall', 'AUC'])


In [13]:
#Entrenamiento del modelo
fc_model.fit(
    x_train, y_train,
    batch_size=32,  #tamaño de lote reducido
    epochs=20,  #mas iteraciones para una mejor convergencia
    validation_data=(x_test, y_test),
    class_weight=class_weights_dict
)

# Evaluar el modelo
test_loss, test_recall, test_auc = fc_model.evaluate(x_test, y_test, verbose=2)
print("Test recall:", test_recall)
print("Test AUC:", test_auc)
print("Test loss:", test_loss)


Epoch 1/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - AUC: 0.5110 - Recall: 0.4653 - loss: 0.8581 - val_AUC: 0.5198 - val_Recall: 0.0859 - val_loss: 0.6776
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - AUC: 0.4899 - Recall: 0.4487 - loss: 0.7191 - val_AUC: 0.5231 - val_Recall: 0.4688 - val_loss: 0.6916
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - AUC: 0.4942 - Recall: 0.6712 - loss: 0.6986 - val_AUC: 0.5087 - val_Recall: 1.0000 - val_loss: 0.6949
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - AUC: 0.4869 - Recall: 0.7689 - loss: 0.6979 - val_AUC: 0.5101 - val_Recall: 0.9531 - val_loss: 0.6963
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - AUC: 0.5035 - Recall: 0.7750 - loss: 0.6902 - val_AUC: 0.4965 - val_Recall: 0.9922 - val_loss: 0.6940
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - AUC: 0.5123 - Recall: 0.9445 - loss: 0.6956 - val_AUC: 0.4973 - val_Recall: 1.0000 - val_loss: 0.6942
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - AUC: 0.5042 - Re